In [572]:
# pip install rich

import rich, math
import numpy as np
import pandas as pd
from rich.console import Console
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, ClassVar
from functools import reduce

In [702]:
BLE_BANDWIDTH = 1000 * 1000 * 2 # b/s
BLE_BANDWIDTH_B_MS = BLE_BANDWIDTH / 8 / 1000 # B/ms
FIRST_PRIMES_BEFORE_200 = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]

@dataclass
class Packet:
    mtu: int = 256 # B

    @property
    def toa(self):
        return self.mtu / BLE_BANDWIDTH_B_MS

PACKET = Packet()
TIME_PADDING = 1 # ms

SLOT_INFO = {
    "empty": {
        "direction": "n/a",
        "is_shared": False,
        "abbreviation": "E",
        "color": "white",
    },
    "data-down": {
        "direction": "down",
        "is_shared": True,
        "abbreviation": "D",
        "color": "green",
    },
    "data-up": {
        "direction": "up",
        "is_shared": False,
        "abbreviation": "U",
        "color": "yellow",
    },
    "beacon": {
        "direction": "down",
        "is_shared": True,
        "abbreviation": "B",
        "color": "red",
    },
    "join-request": {
        "direction": "up",
        "is_shared": True,
        "abbreviation": "J",
        "color": "purple",
    },
    "join-response": {
        "direction": "down",
        "is_shared": True,
        "abbreviation": "R",
        "color": "deep_pink3",
    },
}

@dataclass
class Slot:
    type: str
    start: Optional[float] = None

    def __post_init__(self):
        assert self.type in SLOT_INFO, f"Unknown slot type: {self.type}"

    @staticmethod
    def from_abbreviation(abbrev):
        for type, info in SLOT_INFO.items():
            if info["abbreviation"] == abbrev:
                return Slot(type)
        raise ValueError(f"Unknown slot abbreviation: {ord(abbrev)}")

    @property
    def direction(self):
        return SLOT_INFO[self.type]["direction"]

    @property
    def is_shared(self):
        return SLOT_INFO[self.type]["is_shared"]

    @property
    def duration(self):
        return PACKET.toa + TIME_PADDING

    @property
    def end(self):
        return self.start + self.duration

    def __str__(self):
        return f"{self.type} slot, from {self.start:.2f} to {self.end:.2f} ({PACKET.mtu} B Packet)"
    
    def __repr__(self):
        return self.__str__()

    def repr_nice(self, compact=False):
        """
        Return a single character with rich-based background color (based on type).
        """
        abbrev = SLOT_INFO[self.type]['abbreviation']
        color = SLOT_INFO[self.type]["color"]
        direction = "↓" if self.direction == "down" else " " # invisible space!!
        if compact:
            # return rich.text.Text(abbrev, style=f'bold {color} on {color}')
            if self.direction == "down":
                return rich.text.Text("↓", style=f'bold white on {color}')
            return rich.text.Text(abbrev, style=f'bold {color} on {color}')
        else:
            # add invisible []'s because space doesn't work well with newlines
            hidden_left = rich.text.Text("[", style=f'bold {color} on {color}')
            # hidden_right = rich.text.Text("]", style=f'bold {color} on {color}')
            if self.direction == "down":
                hidden_right = rich.text.Text("↓", style=f'bold white on {color}')
            else:
                hidden_right = rich.text.Text("]", style=f'bold {color} on {color}')
            chr = rich.text.Text(abbrev, style=f'bold white on {color}')
            return hidden_left + chr + hidden_right

class SlotFactory:
    def beacon(n=3):
        return [Slot("beacon") for _ in range(n)]

    def join_request(n):
        return [Slot("join-request") for _ in range(n)]

    def data_down(n):
        return [Slot("data-down") for _ in range(n)]

    def data_up(n):
        return [Slot("data-up") for _ in range(n)]

@dataclass
class Gateway:
    max_nodes: int = 80

GATEWAY = Gateway()

@dataclass
class Slotframe:
    slots: list[Slot]

    @staticmethod
    def from_nested(nested_slots):
        """Creates a Slotframe from nested slots, supporting both lists and single Slot objects."""
        # Use reduce to flatten nested structures automatically
        def flatten(acc, item):
            if isinstance(item, list):
                return acc + item
            return acc + [item]
        
        slots = reduce(flatten, nested_slots if isinstance(nested_slots, list) else [nested_slots], [])
        return Slotframe(slots)

    @staticmethod
    def build(abbreviations, start=0):
        """Creates a Slotframe from a string of abbreviations, e.g., 'BDDUJ' """
        slots = [Slot.from_abbreviation(abbrev) for abbrev in abbreviations if abbrev != " " and abbrev != "\n"]
        for slot in slots:
            slot.start = start
            start += slot.duration
        return Slotframe(slots)

    @staticmethod
    def build_blocks(assoc_slots, data_slots, repetitions=1):
        """Creates a Slotframe from a string of abbreviations, e.g., 'BDDUJ' """
        abbreviations = (assoc_slots + data_slots) * repetitions
        return Slotframe.build(abbreviations)

    @staticmethod
    def build_blocks2(data_slots, repeats_data=1, assoc_slots="BJ", repeats_assoc=1):
        """Creates a Slotframe from a string of abbreviations, e.g., 'BDDUJ' """
        abbreviations = (assoc_slots + data_slots * repeats_data) * repeats_assoc
        return Slotframe.build(abbreviations)

    def __str__(self):
        return f"Slotframe: {len(self.slots)} slots"

    @property
    def slot_duration(self):
        return self.slots[0].duration

    @property
    def duration(self):
        return round(len(self.slots) * self.slot_duration, 2)
    
    @property
    def start(self):
        return self.slots[0].start
    
    @property
    def end(self):
        return self.slots[-1].end
    
    def as_raw_abbreviations(self):
        return "".join([SLOT_INFO[slot.type]["abbreviation"] for slot in self.slots])

    def count_slots_per_type(self, abbreviations=False):
        """Count the number of slots per type."""
        counts = {}
        for slot in self.slots:
            if slot.type not in counts:
                counts[slot.type] = 1
            else:
                counts[slot.type] += 1
        if abbreviations:
            return {SLOT_INFO[type]["abbreviation"]: count for type, count in counts.items()}
        return counts

    def max_nodes(self):
        """Return the maximum number of nodes that can be scheduled in the slotframe."""
        return self.count_slots_per_type().get("data-up", 0)
    
    def ratio_data_up_to_data_down(self):
        return round(self.count_slots_per_type().get("data-up", 0) / self.count_slots_per_type().get("data-down", 1), 2)

    def closest_prime_number(self):
        """Find the closest prime number to the number of slots."""
        return min(FIRST_PRIMES_BEFORE_200, key=lambda x: abs(x - len(self.slots)))
    
    def find_first_slot(self, type):
        """Find the first slot of a given type, including index."""
        return next(((i, slot) for i, slot in enumerate(self.slots) if slot.type == type), (None, None))
    
    def find_last_slot(self, type):
        """Find the last slot of a given type, including index."""
        return next(((j, slot) for j, slot in reversed(list(enumerate(self.slots))) if slot.type == type), (None, None))

    def repr_nice(self, compact=False):
        """Return a rich-based representation of the slotframe."""
        schedule = [slot.repr_nice(compact) for slot in self.slots]
        return rich.text.Text.assemble(*schedule)
    
    def show(self, compact=False):
        """Show the schedule of the slotframe."""
        console = Console()
        console.print(self.repr_nice(compact), end="\n", soft_wrap=compact)

    def show_stats(self, compact=False, otap_size=10_000):
        """Show scheduling statistics of the slotframe. Reuse the repr_nice method of the Slot class."""
        console = Console()
        schedule = [slot.repr_nice(compact) for slot in self.slots]
        console.print(f"Slotframe with {len(self.slots)} slots of {self.slot_duration:.2f} ms each (total {self.duration:.2f} ms):")
        # console.print(*schedule, end="\n")
        console.print(rich.text.Text.assemble(*schedule), end="\n", soft_wrap=compact)
        console.print(f"""\
- Max nodes: {self.max_nodes()} (number of data-up slots)

- MTU per slot: {PACKET.mtu} B
- Number of slots per type: {self.count_slots_per_type()}
- Ratio of data-up to data-down slots: {self.ratio_data_up_to_data_down()}
- Closest prime number: {self.closest_prime_number()}
""")
        return self


# example usage
sf = Slotframe.build_blocks2(data_slots="DUUU", repeats_data=1, assoc_slots="BJ", repeats_assoc=3)
sf.show_stats(compact=False)

Slotframe with 18 slots of 2.02 ms each (total 36.43 ms):

[B↓[J][D↓[U][U][U][B↓[J][D↓[U][U][U][B↓[J][D↓[U][U][U]

- Max nodes: 9 (number of data-up slots)

- MTU per slot: 256 B
- Number of slots per type: {'beacon': 3, 'join-request': 3, 'data-down': 3, 'data-up': 9}
- Ratio of data-up to data-down slots: 3.0
- Closest prime number: 17

Slotframe(slots=[beacon slot, from 0.00 to 2.02 (256 B Packet), join-request slot, from 2.02 to 4.05 (256 B Packet), data-down slot, from 4.05 to 6.07 (256 B Packet), data-up slot, from 6.07 to 8.10 (256 B Packet), data-up slot, from 8.10 to 10.12 (256 B Packet), data-up slot, from 10.12 to 12.14 (256 B Packet), beacon slot, from 12.14 to 14.17 (256 B Packet), join-request slot, from 14.17 to 16.19 (256 B Packet), data-down slot, from 16.19 to 18.22 (256 B Packet), data-up slot, from 18.22 to 20.24 (256 B Packet), data-up slot, from 20.24 to 22.26 (256 B Packet), data-up slot, from 22.26 to 24.29 (256 B Packet), beacon slot, from 24.29 to 26.31 (256 B Packet), join-request slot, from 26.31 to 28.34 (256 B Packet), data-down slot, from 28.34 to 30.36 (256 B Packet), data-up slot, from 30.36 to 32.38 (256 B Packet), data-up slot, from 32.38 to 34.41 (256 B Packet), data-up slot, from 34.41 to 36.43 (256 B Packet)])

In [627]:

@dataclass
class Latency():
    sf: Slotframe

    def worst_case_downlink(self):
        """The diff between the last uplink and the first downlink."""
        i, first_down = self.sf.find_first_slot("data-down")
        j, last_up = self.sf.find_last_slot("data-up")
        return round(last_up.end - first_down.start, 2)
    def worst_case_uplink_estimate(self):
        """
        Just an estimate for the worst case latency, when all nodes are waiting for a downlink reply.

        - this is basically using the ratio of uplink to downlink slots, and assuming that we are the last in queue to receive a downlink reply.
        - this only makes sense for potential scenarios where the robot wants to perform a request-response style communication, e.g., EDHOC handshake.
        """
        sf_up = self.sf.count_slots_per_type()["data-up"]
        sf_down = self.sf.count_slots_per_type()["data-down"]
        needed_sf_n = math.ceil(sf_up / sf_down)
        return round(needed_sf_n * self.sf.duration, 2)
    
    def show_stats(self):
        console = Console()
        console.print(f"""\
Latency worst case:

- downlink/uplink reply:            {self.worst_case_downlink()} ms
- uplink/downlink reply (estimate): {self.worst_case_uplink_estimate()} ms
    """)
    
Latency(sf).show_stats()

Latency worst case:

- downlink/uplink reply:            18.22 ms
- uplink/downlink reply (estimate): 48.58 ms

In [575]:
@dataclass
class OTAP:
    sf: Slotframe

    def duration(self, size=10_000):
        return self.get_stats(size)["duration"]

    def get_stats(self, size=10_000):
        swarmit_chunk_size = 128 # B
        packets = math.ceil(size / swarmit_chunk_size)
        data_down_slots = self.sf.count_slots_per_type()["data-down"]
        needed_slotframes = math.ceil(packets / data_down_slots)
        return {
            "image_size": math.ceil(size / 1024),
            "duration": round(needed_slotframes * self.sf.duration, 2),
            "packets": packets,
            "swarmit_chunk_size": swarmit_chunk_size,
            "needed_slotframes": needed_slotframes,
        }
        
    def show_stats(self, size=10_000):
        """Calculate the duration of an OTAP session."""
        stats = self.get_stats(size)
        console = Console()
        console.print(f"""\
OTAP update:

- Image size: {math.ceil(size / 1024)} kB
- Duration: {round(stats["duration"] / 1000, 2)} s

- Packets: {stats["packets"]}
- swarmit_chunk_size: {stats["swarmit_chunk_size"]} B
- Needed slotframes: {stats["needed_slotframes"]}
    """)

# Testing different schedules

In [697]:
@dataclass
class SFConfigs:
    sf_configs: list[Slotframe]
    otap_size: int = 10_000

    def get_df(self):
        headers = ["max_nodes", "down_slots", "ratio_up_down", "sf_duration", "max_latency_down", "max_est_latency_up", "otap_duration"]
        stats = [
            [
                sf.max_nodes(),
                sf.count_slots_per_type()["data-down"],
                sf.ratio_data_up_to_data_down(),
                sf.duration,
                Latency(sf).worst_case_downlink(),
                Latency(sf).worst_case_uplink_estimate(),
                OTAP(sf).duration(self.otap_size),
            ]
            for sf in self.sf_configs
        ]

        return pd.DataFrame(stats, columns=headers)

    def show_df(self):
        display(self.get_df())

    def show_schedules(self, compact=True):
        """Display compact slotframes with prepended indices and newlines."""
        texts = []
        for i, sf in enumerate(self.sf_configs):
            # Prepend index and avoid extra spaces
            index_text = rich.text.Text(f"{i}: ", style="bold magenta", end="")
            slotframe_text = sf.repr_nice(compact=compact)
            newline = rich.text.Text("\n", end="")
            texts.append(rich.text.Text.assemble(index_text, slotframe_text, newline))

        console = Console()
        # Use '\n' to join the lines explicitly for precise layout
        combined_text = rich.text.Text("\n").join(texts)
        console.print(combined_text, end="", soft_wrap=True)

    def show_stats(self, compact=True):
        """Display statistics for all slotframes."""
        self.show_df()
        self.show_schedules(compact)

# example usage
sf_configs = SFConfigs([
    Slotframe.build_blocks2("DUU", 3),
], otap_size=30_000).show_stats()

,max_nodes,down_slots,ratio_up_down,sf_duration,max_latency_down,max_est_latency_up,otap_duration
0,6,3,2.0,22.26,18.22,44.52,1758.54


0: ↓ ↓  ↓  ↓  

In [703]:
sf_configs = SFConfigs([
    Slotframe.build_blocks2("DUUUUUUUUUUUUJ", 9, "BJ"),
    Slotframe.build_blocks2("DUUUUUUUJ", 12, "BJ"),
    Slotframe.build_blocks2("DUUUJDUUU", 12, "BJ"),
    Slotframe.build_blocks2("DUDUDJDUDU", 12, "BJ"),
    Slotframe.build_blocks2("DUUUUUUUUUUUUJ", 3, "BJ", repeats_assoc=3),
    Slotframe.build_blocks2("DUUUUUUUJ", 4, "BJ", repeats_assoc=3),
    Slotframe.build_blocks2("DUUUJDUUU", 4, "BJ", repeats_assoc=3),
    Slotframe.build_blocks2("DUDUDJDUDU", 4, "BJ", repeats_assoc=3),
], otap_size=30_000).show_stats()

,max_nodes,down_slots,ratio_up_down,sf_duration,max_latency_down,max_est_latency_up,otap_duration
0,108,9,12.0,259.07,253.00,3108.84,6994.89
1,84,12,7.0,222.64,216.57,1558.48,4452.80
2,72,24,3.0,222.64,218.59,667.92,2226.40
3,48,60,0.8,246.93,242.88,246.93,987.72
4,108,9,12.0,267.17,261.10,3206.04,7213.59
5,84,12,7.0,230.74,224.66,1615.18,4614.80
6,72,24,3.0,230.74,226.69,692.22,2307.40
7,48,60,0.8,255.02,250.98,255.02,1020.08


0: ↓J↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ

1: ↓J↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ

2: ↓J↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU

3: ↓J↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U

4: ↓J↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓J↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓J↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ↓UUUUUUUUUUUUJ

5: ↓J↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓J↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓J↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ↓UUUUUUUJ

6: ↓J↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓J↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓J↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU↓UUUJ↓UUU

7: ↓J↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓J↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓J↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U↓U↓U↓J↓U↓U

# Scratchpad

In [576]:
sf = Slotframe.build_blocks(
    "BJR", # slots for association: beacon, join-request, join-response
    "DUUUUUUUU" * 4, # slots for data: one data-down, several data-up (repeat a few times)
    # "DUDUDUDU" * 4,
    3, # repeat pattern 3 times within the slotframe, each Beacon slot uses a different BLE advertising channel
).show_stats()

Latency(sf).show_stats()
OTAP(sf).show_stats(size=30_000)

Slotframe with 117 slots of 2.02 ms each (total 236.81 ms):

[B][J][R][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][
U][B][J][R][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U
][U][B][J][R][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U][U][U][D][U][U][U][U][U][U]
[U][U]

- Max nodes: 96 (number of data-up slots)

- MTU per slot: 256 B
- Number of slots per type: {'beacon': 3, 'join-request': 3, 'join-response': 3, 'data-down': 12, 'data-up': 96}
- Ratio of data-down to data-up slots: 0.12
- Closest prime number: 113

Latency worst case:

- downlink/uplink reply:            230.74 ms
- uplink/downlink reply (estimate): 1894.48 ms

OTAP update:

- Image size: 30 kB
- Duration: 4.74 s

- Packets: 235
- swarmit_chunk_size: 128 B
- Needed slotframes: 20

In [577]:
sf = Slotframe.build_blocks2(
    # "DUUUUUUUU", 12, # slots for data: one data-down, several data-up (repeat a few times)
      "DUUDUUDUU", 12, # a bit more data down
).show_stats()

Latency(sf).show_stats()
OTAP(sf).show_stats(size=30_000)

Slotframe with 111 slots of 2.02 ms each (total 224.66 ms):

[B][J][R][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][
U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U
][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U][D][U][U]

- Max nodes: 72 (number of data-up slots)

- MTU per slot: 256 B
- Number of slots per type: {'beacon': 1, 'join-request': 1, 'join-response': 1, 'data-down': 36, 'data-up': 72}
- Ratio of data-down to data-up slots: 0.5
- Closest prime number: 109

Latency worst case:

- downlink/uplink reply:            218.59 ms
- uplink/downlink reply (estimate): 449.32 ms

OTAP update:

- Image size: 30 kB
- Duration: 1.57 s

- Packets: 235
- swarmit_chunk_size: 128 B
- Needed slotframes: 7